In [1]:
import os, sys
from os import listdir
from os.path import isfile, join, isdir
import numpy as np
import matplotlib.pyplot as plt
import torch

sys.path.append('/home/vanveen/ConvDecoder/')
from utils.data_io import load_h5_qdess
from utils.transform import reshape_adj_channels_to_complex_vals, \
                            reshape_complex_vals_to_adj_channels

In [2]:
from include.decoder_conv import init_convdecoder
from include.fit import fit
from utils.transform import ifft_2d, fft_2d
from include.mri_helpers import apply_mask

torch.cuda.set_device(2)

# TODO
- build empirical loss adjustment per akshay rec
    - make sure we are penalizing e1, e2 similarly
    - what is the loss for e1, e2 separately? print these in ipynb
        - if similar, then network is expressing each equally
        - if not, want to re-balance these w empirical adjustment
    - figure out what this means in terms of t2 maps...

In [6]:
file_id = '005'
ksp = load_h5_qdess(file_id)

# load, concat both echo slices
idx_kx = ksp.shape[0] // 2 # want central slice in kx (axial) b/c we undersample in (ky,kz)
ksp_echo1 = ksp[:,:,:,0,:].permute(3,0,1,2)[:, idx_kx, :, :]
ksp_echo2 = ksp[:,:,:,1,:].permute(3,0,1,2)[:, idx_kx, :, :]
ksp_orig = torch.cat((ksp_echo1, ksp_echo2), 0)

In [7]:
print(ksp_orig.shape)

torch.Size([16, 512, 160])


In [ ]:
net, net_input, ksp_orig_ = init_convdecoder(ksp_orig)

ksp_masked, mask = apply_mask(ksp_orig_, accel=4)
im_masked = ifft_2d(ksp_masked)

In [ ]:
net = fit(ksp_masked=ksp_masked, img_masked=im_masked,
                          net=net, net_input=net_input, mask=mask)